<a href="https://colab.research.google.com/github/imchukwu/movie_recommendation_system/blob/main/recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19.

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0   979k      0 --:--:-- --:--:-- --:--:--  979k


***Recommendation System - Collaborative Filter + Matrix Factorization & KMeans***


In [2]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [3]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [4]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [5]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [8]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)

    def predict(self, user, item):
        return self.forward(user, item)

In [9]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()

        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()

        #--- Producing new continuous IDs for users and movies ---

        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}

        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])


        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [10]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0444, 0.0325, 0.0102,  ..., 0.0164, 0.0081, 0.0436],
        [0.0205, 0.0455, 0.0111,  ..., 0.0110, 0.0405, 0.0049],
        [0.0177, 0.0460, 0.0145,  ..., 0.0209, 0.0337, 0.0257],
        ...,
        [0.0313, 0.0071, 0.0394,  ..., 0.0468, 0.0377, 0.0009],
        [0.0407, 0.0397, 0.0319,  ..., 0.0260, 0.0214, 0.0370],
        [0.0401, 0.0322, 0.0279,  ..., 0.0295, 0.0098, 0.0215]])
item_factors.weight tensor([[0.0215, 0.0428, 0.0015,  ..., 0.0288, 0.0202, 0.0390],
        [0.0476, 0.0316, 0.0331,  ..., 0.0388, 0.0340, 0.0314],
        [0.0369, 0.0163, 0.0417,  ..., 0.0256, 0.0029, 0.0135],
        ...,
        [0.0054, 0.0157, 0.0162,  ..., 0.0492, 0.0076, 0.0031],
        [0.0238, 0.0013, 0.0301,  ..., 0.0088, 0.0097, 0.0058],
        [0.0126, 0.0198, 0.0025,  ..., 0.0089, 0.0325, 0.0112]])


In [11]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

<ipython-input-11-dad152416852>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.077219662932576
iter #1 Loss: 4.753596235350304
iter #2 Loss: 2.4752201410719588
iter #3 Loss: 1.721058018240832
iter #4 Loss: 1.345240585241221
iter #5 Loss: 1.1283204314974964
iter #6 Loss: 0.9913494183781183
iter #7 Loss: 0.9001198075900828
iter #8 Loss: 0.8368619749237438
iter #9 Loss: 0.7921715466125967
iter #10 Loss: 0.7590356516051413
iter #11 Loss: 0.7344835692553351
iter #12 Loss: 0.7159534454118782
iter #13 Loss: 0.7017543836671689
iter #14 Loss: 0.6902290396959648
iter #15 Loss: 0.6815989484566117
iter #16 Loss: 0.6751747949398713
iter #17 Loss: 0.6696315397163333
iter #18 Loss: 0.66593858641265
iter #19 Loss: 0.6626500155644368
iter #20 Loss: 0.6605621740070696
iter #21 Loss: 0.6589234027311887
iter #22 Loss: 0.6576260539707799
iter #23 Loss: 0.6566855190369079
iter #24 Loss: 0.6559629105220591
iter #25 Loss: 0.6552902767954744
iter #26 Loss: 0.6542335432873765
iter #27 Loss: 0.6534576193632813
iter #28 Loss: 0.6524002079658097
iter #29 Loss: 0.651695686472

In [12]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[ 1.1027,  2.2764,  0.9192,  ...,  1.6449,  0.5396,  1.0370],
        [ 0.0651,  0.4746, -0.1261,  ...,  2.3743,  0.9270,  1.0615],
        [-2.1401,  0.6833,  0.4427,  ...,  1.8588,  2.6410, -1.7490],
        ...,
        [ 1.2135,  0.1418,  1.6689,  ...,  0.9731,  2.7214, -0.4581],
        [ 1.3587,  1.1958,  0.5301,  ...,  1.2966,  1.0729,  0.9513],
        [ 0.5665,  1.4634,  1.2564,  ...,  0.8954,  0.6217,  0.7158]],
       device='cuda:0')
item_factors.weight tensor([[0.3713, 0.5691, 0.2589,  ..., 0.2542, 0.3169, 0.7366],
        [0.6521, 0.4518, 0.4184,  ..., 0.7602, 0.1490, 0.8826],
        [0.7192, 0.4469, 0.4600,  ..., 0.1477, 0.3404, 0.5123],
        ...,
        [0.3244, 0.3303, 0.3314,  ..., 0.3651, 0.3226, 0.3175],
        [0.4206, 0.3937, 0.4237,  ..., 0.4011, 0.4046, 0.4006],
        [0.4153, 0.4222, 0.4060,  ..., 0.4134, 0.4358, 0.4124]],
       device='cuda:0')


In [13]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [14]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [15]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [16]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
    print("Cluster #{}".format(cluster))
    movs = []
    # Find movie indices belonging to the current cluster
    for movidx in np.where(kmeans.labels_ == cluster)[0]:
        movid = train_set.idx2movieid[movidx]
        # Check how many ratings this movie has
        rat_count = len(ratings_df.loc[ratings_df['movieId'] == movid])
        movs.append((movie_names[movid], rat_count))
    # Sort movies by rating count in descending order and print top 10
    for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
        print("\t", mov[0])

Cluster #0
	 Independence Day (a.k.a. ID4) (1996)
	 True Lies (1994)
	 Speed (1994)
	 Mission: Impossible (1996)
	 Mask, The (1994)
	 Mrs. Doubtfire (1993)
	 Star Wars: Episode I - The Phantom Menace (1999)
	 Batman Forever (1995)
	 Pretty Woman (1990)
	 X-Men (2000)
Cluster #1
	 Jurassic Park (1993)
	 Terminator 2: Judgment Day (1991)
	 Apollo 13 (1995)
	 Fugitive, The (1993)
	 Batman (1989)
	 Saving Private Ryan (1998)
	 Sixth Sense, The (1999)
	 Lion King, The (1994)
	 Gladiator (2000)
	 Shrek (2001)
Cluster #2
	 Batman & Robin (1997)
	 Scream 3 (2000)
	 Buffy the Vampire Slayer (1992)
	 Toys (1992)
	 Honey, I Blew Up the Kid (1992)
	 Superman IV: The Quest for Peace (1987)
	 Karate Kid, Part III, The (1989)
	 Last Man Standing (1996)
	 Highlander III: The Sorcerer (a.k.a. Highlander: The Final Dimension) (1994)
	 Rambo III (1988)
Cluster #3
	 Dumb & Dumber (Dumb and Dumber) (1994)
	 Net, The (1995)
	 Avatar (2009)
	 Ace Ventura: When Nature Calls (1995)
	 RoboCop (1987)
	 Pleasantv